In [47]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.cross_validation import train_test_split

In [48]:
train = pd.read_csv('input/train.csv')
test = pd.read_csv('input/test.csv')

In [49]:
train_no_id = train.drop('id',axis=1)
train70, train30 = train_test_split(train_no_id,test_size=0.3,random_state=192)

In [50]:
with tf.name_scope('logres'):
    x = tf.placeholder(tf.float32, [None, 57])
    W = tf.Variable(tf.zeros([57, 1]))
    b = tf.Variable(tf.zeros([1]))
    y = tf.matmul(x, W)+b
    y_ = tf.placeholder(tf.float32, [None, 1])

In [51]:
with tf.name_scope('loss'):
    cross_entropy = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=y_, logits=y))
    train_step = tf.train.GradientDescentOptimizer(0.005).minimize(cross_entropy)

In [52]:
with tf.name_scope('accuracy'):
    accuracy_op = tf.metrics.auc(y_,tf.sigmoid(y))

In [53]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for j in range(5):
        for i in range(100):
            train_batch = train70.sample(frac=0.001).reset_index(drop=True)
            batch_xs = train_batch.drop('target',axis=1)
            batch_ys = train_batch['target'].values.reshape(train_batch.shape[0],1)
            sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})
            
        sess.run(tf.local_variables_initializer())
        print(sess.run(accuracy_op,feed_dict={x: train70.drop('target',axis=1), y_: train70['target'].values.reshape(train70.shape[0],1)})) 
    

(0.0, 0.49150965)
(0.0, 0.50404871)
(0.0, 0.51595974)
(0.0, 0.52095807)
(0.0, 0.51513648)
